In [32]:
from iapws import IAPWS97
import math

times = 1e-3 ##days
time  = times *24*3600



g_G = 0.035 # K/m, geothermal gradient
Teio = 295 # K, undisturbed ground temperature

Tinj = 283.5
Ts = 283.5
t = 3600*24*times
z = 3000

T_ei = Teio + (g_G * z)


'''water properties'''
water = IAPWS97(T = 283.5, P = 2)
rho_f = water.rho
c_f = water.cp * 1e3
k_f = water.k
mu_f = water.mu


'''rock properties'''
# alpha_e = 1e-6
rho_e = 1800
c_e = 1000
k_e = 1.8
alpha_e = k_e / (rho_e * c_e)

'''outer tubing'''
d_in = 0.08 #Insulation outside diameter, m
r_in = 0.5 * d_in
d_ci = 0.13 #Casing inside diameter, m
r_ci = 0.5 * d_ci
d_co = 0.14 #Casing outside diameter, m
r_co = d_co * 0.5
d_cem = 0.24 #Cement outside diameter, m
r_cem = 0.5 * d_cem
r_wb = r_cem

'''thermal conductivities'''
k_c = 57 #Tubing and casing steel thermal conductivity, Wm-1K-1
k_t = k_c
k_cem = 2.3 #Cement thermal conductivity, Wm-1K-1
k_in = 0.023 #Insulation thermal conductivity, Wm-1K-1
rough_tc = 0.0003 #Tubing and casing steel absolute roughness, m


'''Wellbore dimensions'''
d_in = 0.08 #Insulation outside diameter, m
r_in = 0.5 * d_in
d_ci = 0.13 #Casing inside diameter, m
r_ci = 0.5 * d_ci

'''inner pipe'''
'''radius/diameters'''
r_e = 100 #formation radius, m
d_ti = 0.06 #Inner tubing inside diameter, m
r_ti = d_ti / 2
d_ito = 0.07 #Inner tubing outside diameter, m
r_ito = d_ito / 2
r_to = r_ito

d_e1 = 2 * r_ti # equivalent diameter, m
d_e2 = 2 * math.sqrt((r_ci * r_ci) - (r_in * r_in)) # equivalent diameter, m


'''Areas of cross-section'''
A_i = math.pi * ((r_ci * r_ci) - (r_in * r_in)) # cross-section area of injection tubing, m2
A_p = math.pi * r_ti * r_ti # cross-section area of production tubing, m2


'''velocities'''
v_fi = 0.2 # injection fluid velocity, m s-1
w = A_i * rho_f * v_fi
v_fp = w / (A_p * rho_f)


'''Coefficients of heat transfer'''
Re1 = rho_f * v_fp * d_e1 / mu_f
Re2 = rho_f * v_fi * d_e2 / mu_f
Pr = mu_f * c_f / k_f

h_f1 = 0.023 * k_f * (Re1**0.8) * (Pr**0.4) / d_e1
h_f2 = 0.023 * k_f * (Re2**0.8) * (Pr**0.4) / d_e2

U_co = 1 / ((r_co / (r_ci * h_f2))         + ((r_co * math.log(r_co / r_ci)) / k_c)         + (((r_co * math.log(r_wb / r_co)) / k_cem)))

U_to = 1 / ((r_to / (r_ti * h_f1)) + ((r_to * math.log(r_to / r_ti)) / k_t) + ((r_to * math.log(r_in / r_to)) / k_in)            + (r_to / (r_in * h_f2)))

t_D = (alpha_e * time) / (r_wb * r_wb) # alpha_e is same as k_e / rho_e * c_e
T_D = math.log((math.exp(-0.2 * t_D)) + ((1.5 - (0.3719 * math.exp(-t_D))) * math.sqrt(t_D)))


# In[42]:


'''defining constants'''

r = (2 * math.pi*r_co*U_co*k_e * A_i*rho_f*c_f)/(k_e + r_co*U_co*T_D)
p = v_fi
m = v_fp
n = (2*math.pi*r_to*U_to)/(A_p*rho_f*c_f)
q = -n




'''evaluating m1 and m2'''


m1 = -0.5*(n-q+r) + 0.5 * math.sqrt((n-q+r)**2- 4*n*r) ## note here : the sqrt term is coming out neg. rn
m2 = -0.5*(n-q+r) - 0.5 * math.sqrt((n-q+r)**2- 4*n*r)

# print(q, r, n)


# In[30]:


'''evaluating A and B'''

B =  m1*(Tinj - Ts)/(m1-m2)
A = -m2*(Tinj - Ts)/(m1-m2)


# In[31]:


'''now getting the production temperature'''
Tp = A*math.exp(m1*t) + B*math.exp(m2*t) + Ts + g_G*(z-m*t)

'''getting the injection temperature'''
Ti = (m1/n + 1)* A * math.exp(m1*t) + (m2/n + 1) * B*math.exp(m2*t) + Ts + g_G*(z-p*t)





h1 = (q - r)*(m1/n + 1) - q
h2 = (m1/n + 1)*m1
b1 = (q-r)*(m2/n + 1) - q
b2 = (m2/n + 1)*m2
print(h1-h2)
print(b1-b2)
print(Tp)
print(Ti)
print('%0.2f' %(Tp-273.15))
print('%0.2f' %(Ti-273.15))

-0.1173372596253432
0.0
386.736
387.8952
113.59
114.75
